Set Shots - Expected Goal Model - Model Tuning and Building - GBM

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import set_shots_file_path
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [3]:
model_version = 6
model_name = 'expected_goal_set'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [4]:
RESPONSE = ModellingDataContract.RESPONSE_GOAL

In [5]:
FEATURES = ModellingDataContract.set_goal_modelling_feature_list

Load Data

In [6]:
df_modelling = pd.read_csv(set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
11052,208,possGain,behind,1663.0,4,634,683.0,Geelong,Geelong,Tom Hawkins,Tom_Hawkins,Kick,41.0,25.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,1,0,105.0,32.0,73.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,48.0,48.0,26.0,25.0,21.0,21.0,14.0,683.0,644.0,642.0,641.0,49.0,39.0,8.062258,39.0,25.0,46.324939,0.570040,32.660913,0.116867,6.695990,0.364585,20.889203,1521.0,3.663562,True,43,True
11053,212,kickIn,goal,1730.0,4,787,885.0,Geelong,Geelong,Patrick Dangerfield,Patrick_Dangerfield,Kick,51.0,43.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,107.0,33.0,74.0,Kick,Contested Mark,Contest Target,Kick Inside 50 Result,51.0,53.0,53.0,53.0,43.0,38.0,38.0,38.0,885.0,852.0,851.0,850.0,98.0,33.0,5.385165,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,29,True
11054,213,centreBounce,goal,1754.0,4,956,979.0,Sydney,Sydney,Paddy McCartin,Paddy_McCartin,Kick,66.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,119.0,33.0,86.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,66.0,74.0,74.0,27.0,0.0,0.0,0.0,17.0,979.0,977.0,968.0,967.0,23.0,2.0,8.000000,14.0,0.0,14.000000,0.000000,0.000000,0.482343,27.636207,2.588598,148.315731,196.0,2.639057,True,78,True
11055,219,centreBounce,goal,1846.0,4,1252,1266.0,Sydney,Sydney,Chad Warner,Chad_Warner,Kick,26.0,3.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,125.0,39.0,86.0,Kick,Uncontested Mark,Ground Kick,Handball,26.0,18.0,-5.0,-2.0,3.0,2.0,-4.0,3.0,1266.0,1265.0,1264.0,1260.0,14.0,1.0,8.062258,54.0,3.0,54.083269,0.055499,3.179830,0.118569,6.793500,0.365993,20.969862,2916.0,3.988984,True,53,True
11056,228,centreBounce,goal,1912.0,4,1519,1560.0,Geelong,Geelong,Jeremy Cameron,Jeremy_Cameron,Kick,41.0,21.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,131.0,45.0,86.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,49.0,49.0,25.0,21.0,17.0,17.0,33.0,1560.0,1535.0,1532.0,1531.0,41.0,25.0,8.944272,39.0,21.0,44.294469,0.493941,28.300756,0.127885,7.327246,0.400462,22.944787,1521.0,3.663562,True,66,True


In [7]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])
# event_type_map = {
#     'Uncontested Mark':'uncontestedMark',
#     'Contested Mark':'contestedMark',
#     'Mark On Lead':'uncontestedMark',
#     'Free For':'freeFor',
#     'Free For: In Possession':'freeFor', 
#     'Pack Mark (P)':'contestedMark', 
#     'Free Advantage':'freeFor',
#     'Free For: Off The Ball':'freeFor',
#     'Mark Fumbled':'contestedMark',
#     'Mark Dropped':'contestedMark',
#     'Free For: Before the Bounce':'freeFor',
#     'Pack Mark (O)':'contestedMark'
# }
# df_modelling["Event_Type1"] = df_modelling['Event_Type1'].replace(event_type_map)
# df_modelling[['uncontestedMark', 'contestedMark', 'freeFor']] = pd.get_dummies(df_modelling['Event_Type1'])

In [8]:
training_data = df_modelling[df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = df_modelling[~df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [9]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [10]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter,ballUp,centreBounce,kickIn,possGain,throwIn
11,122,possGain,goal,1074.0,3,28,72.0,Sydney,Sydney,Nick Blakey,Nick_Blakey,Kick,28.0,-14.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,0,6,51.0,52.0,-1.0,Kick,Uncontested Mark,Kick Inside 50 Result,Kick Into F50,28.0,35.0,35.0,32.0,-14.0,-12.0,-12.0,35.0,72.0,38.0,36.0,36.0,44.0,34.0,7.280110,50.0,14.0,51.923020,0.273009,15.642246,0.119147,6.826615,0.368686,21.124166,2500.0,3.912023,True,83,False,0,0,0,1,0
12,132,possGain,goal,1168.0,3,422,469.0,Sydney,Sydney,Logan McDonald,Logan_McDonald,Kick,47.0,-4.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,0,6,57.0,58.0,-1.0,Kick,Mark On Lead,Kick,Gather,47.0,55.0,30.0,22.0,-4.0,-4.0,2.0,6.0,469.0,434.0,432.0,430.0,47.0,35.0,8.000000,31.0,4.0,31.256999,0.128323,7.352379,0.205222,11.758330,0.672664,38.540807,961.0,3.433987,True,93,False,0,0,0,1,0
15,144,throwIn,goal,1283.0,3,1014,1041.0,Sydney,Sydney,Callum Mills,Callum_Mills,Kick,29.0,-3.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,0,6,58.0,71.0,-13.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,29.0,37.0,37.0,22.0,-3.0,-2.0,-2.0,-16.0,1041.0,1019.0,1018.0,1017.0,27.0,22.0,8.062258,49.0,3.0,49.091751,0.061148,3.503532,0.130680,7.487397,0.405895,23.256079,2401.0,3.891820,True,99,False,0,0,0,0,1
20,194,throwIn,goal,1678.0,4,589,654.0,Sydney,Sydney,Luke Parker,Luke_Parker,Kick,49.0,-43.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,6,0,6,71.0,107.0,-36.0,Kick,Uncontested Mark,Kick Inside 50 Result,Kick Into F50,49.0,53.0,53.0,7.0,-43.0,-40.0,-40.0,-51.0,654.0,624.0,622.0,621.0,65.0,30.0,5.000000,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,96,False,0,0,0,0,1
31,84,centreBounce,goal,676.0,2,418,472.0,Western Bulldogs,Western Bulldogs,Lachlan McNeil,Lachlan_McNeil,Kick,71.0,0.0,effective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021,0,0,6,0,6,14.0,26.0,-12.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,71.0,42.0,42.0,7.0,0.0,-21.0,-21.0,-20.0,472.0,430.0,428.0,427.0,54.0,42.0,35.805028,9.0,0.0,9.000000,0.000000,0.000000,0.814019,46.639866,-15.483871,-887.160457,81.0,2.197225,True,95,False,0,1,0,0,0


Preprocess Data

In [11]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [12]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]

In [13]:
X_train_preproc.shape, X_test_preproc.shape

((8749, 5), (2308, 5))

In [14]:
X_train_preproc.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain
0,46.0,52.009614,0.908067,0.075999,0
1,52.0,43.600459,0.931882,0.088007,1
2,45.0,36.674242,0.451453,0.158231,1
3,59.0,33.015148,0.957589,0.112618,0
4,33.0,46.572524,0.260602,0.133410,0


In [15]:
X_test_preproc.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain
11,28.0,51.923020,0.273009,0.119147,1
12,47.0,31.256999,0.128323,0.205222,1
15,29.0,49.091751,0.061148,0.130680,0
20,49.0,51.865210,0.977446,0.069260,0
31,71.0,9.000000,0.000000,0.814019,0


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [16]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints = ModellingDataContract.monotone_constraints_set_goal)

In [17]:
xgb_tuner.training_data.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain
0,46.0,52.009614,0.908067,0.075999,0
1,52.0,43.600459,0.931882,0.088007,1
2,45.0,36.674242,0.451453,0.158231,1
3,59.0,33.015148,0.957589,0.112618,0
4,33.0,46.572524,0.260602,0.133410,0


In [18]:
xgb_tuner.tune_hyperparameters()

[I 2023-07-03 18:18:42,588] A new study created in memory with name: no-name-da20b3b7-0b15-43ae-858c-63d78ab16e8b
[I 2023-07-03 18:18:42,622] Trial 0 finished with value: 0.6045834064671619 and parameters: {'max_depth': 15, 'min_child_weight': 31, 'eta': 0.4888206012203858, 'gamma': 0.00018682050901051264, 'lambda': 0.6328648244592768, 'alpha': 0.011664671392884659, 'subsample': 0.32765221674001754, 'colsample_bytree': 0.6238396136446652}. Best is trial 0 with value: 0.6045834064671619.
[I 2023-07-03 18:18:42,651] Trial 1 finished with value: 0.6564079351024595 and parameters: {'max_depth': 11, 'min_child_weight': 29, 'eta': 0.03248998553200251, 'gamma': 0.0021165786775577187, 'lambda': 0.006948176369292431, 'alpha': 0.005883725478012826, 'subsample': 0.5400584655072955, 'colsample_bytree': 0.5415609836688912}. Best is trial 0 with value: 0.6045834064671619.
[I 2023-07-03 18:18:42,678] Trial 2 finished with value: 0.6097391106614307 and parameters: {'max_depth': 9, 'min_child_weight': 

Number of finished trials:  1000
Best trial:
  Value: 0.5859618890529605
  Params: 
    max_depth: 15
    min_child_weight: 92
    eta: 0.9069170734078557
    gamma: 0.02898806234727525
    lambda: 0.1907222963499189
    alpha: 0.5978234152443184
    subsample: 0.7786933908076759
    colsample_bytree: 0.5548842560471717


In [19]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 15,
 'min_child_weight': 92,
 'eta': 0.9069170734078557,
 'gamma': 0.02898806234727525,
 'lambda': 0.1907222963499189,
 'alpha': 0.5978234152443184,
 'subsample': 0.7786933908076759,
 'colsample_bytree': 0.5548842560471717}

Training Model - SuperXGBClassifier class for training and predictions

In [20]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = ModellingDataContract.monotone_constraints_set_goal

In [21]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test, 
                               params = params)

In [22]:
super_xgb.fit()

[0]	validation_0-logloss:0.61392	validation_1-logloss:0.61432
[1]	validation_0-logloss:0.61362	validation_1-logloss:0.61469
[2]	validation_0-logloss:0.61365	validation_1-logloss:0.61490
[3]	validation_0-logloss:0.61364	validation_1-logloss:0.61500
[4]	validation_0-logloss:0.61185	validation_1-logloss:0.61343
[5]	validation_0-logloss:0.60825	validation_1-logloss:0.61151
[6]	validation_0-logloss:0.60819	validation_1-logloss:0.61083
[7]	validation_0-logloss:0.60809	validation_1-logloss:0.61184
[8]	validation_0-logloss:0.60559	validation_1-logloss:0.61086
[9]	validation_0-logloss:0.60559	validation_1-logloss:0.61047
[10]	validation_0-logloss:0.60570	validation_1-logloss:0.61078
[11]	validation_0-logloss:0.60534	validation_1-logloss:0.61132
[12]	validation_0-logloss:0.60535	validation_1-logloss:0.61155
[13]	validation_0-logloss:0.60538	validation_1-logloss:0.61153
[14]	validation_0-logloss:0.60548	validation_1-logloss:0.61163
[15]	validation_0-logloss:0.60562	validation_1-logloss:0.61204
[1

In [23]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5548842560471717, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.02898806234727525, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.9069170734078557, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=92, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': -1,
                                    'Distance_to_Middle_Goal': -1,
                                    'Visible_Goal_Angle': 1, 'x0': -1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [24]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle',
 'possGain']

In [25]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [26]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]

Check Average Predictions

In [27]:
train_probas.mean(), training_data[RESPONSE].mean(), test_probas.mean(), test_data[RESPONSE].mean()

(0.5404654, 0.5429191907646588, 0.5493049, 0.5355285961871751)

Export model

In [28]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [29]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,17,183.0,0,46.0,52.009614,0.908067,0.075999,0,0,0.243751
1,202101_BrisbaneLions_Sydney,28,295.0,0,52.0,43.600459,0.931882,0.088007,1,0,0.334038
2,202101_BrisbaneLions_Sydney,34,354.0,0,45.0,36.674242,0.451453,0.158231,1,1,0.690334
3,202101_BrisbaneLions_Sydney,53,537.0,0,59.0,33.015148,0.957589,0.112618,0,0,0.384747
4,202101_BrisbaneLions_Sydney,59,578.0,1,33.0,46.572524,0.260602,0.133410,0,1,0.564334


In [30]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,possGain,xgb_preds,xgb_probas
0,202101_BrisbaneLions_Sydney,122,1074.0,1,28.0,51.923020,0.273009,0.119147,1,0,0.398399
1,202101_BrisbaneLions_Sydney,132,1168.0,1,47.0,31.256999,0.128323,0.205222,1,1,0.819298
2,202101_BrisbaneLions_Sydney,144,1283.0,1,29.0,49.091751,0.061148,0.130680,0,0,0.492060
3,202101_BrisbaneLions_Sydney,194,1678.0,1,49.0,51.865210,0.977446,0.069260,0,0,0.243751
4,202101_Collingwood_WesternBulldogs,84,676.0,1,71.0,9.000000,0.000000,0.814019,0,1,0.947190


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")